# 라이브러리

In [1]:
import numpy as np
import torch
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if cuda.is_available() else 'cpu'
path = "/content/drive/MyDrive/21_winter/CIFAR-100/cifar-100-python"
batch_size = 32

Mounted at /content/drive


# 데이터 불러오기

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

meta_data = unpickle(path + "/meta")
train_data = unpickle(path + "/train")
test_data = unpickle(path + "/test")

train_data_data = train_data[b'data']
train_data_data.resize(50000,3,32,32)
train_data_data = np.concatenate((train_data_data,
                                  train_data_data[:,:,:,::-1]))

#train_data_label = list(zip(train_data[b'coarse_labels'],
#                       [i % 5 for i in train_data[b'fine_labels']]))
train_data_label = train_data[b'fine_labels']
train_data_label += train_data_label

test_data_data = test_data[b'data']
#test_data_label = list(zip(test_data[b'coarse_labels'],
#                        [i % 5 for i in test_data[b'fine_labels']]))
test_data_label = test_data[b'fine_labels']

In [3]:
"""
import matplotlib.pyplot as plt
plt.imshow(np.transpose(train_data_data[:,:,:,::-1][2], (1, 2, 0)))
"""

'\nimport matplotlib.pyplot as plt\nplt.imshow(np.transpose(train_data_data[:,:,:,::-1][2], (1, 2, 0)))\n'

# 데이터 준비

In [4]:
class GH_Dataset_train(data.Dataset): 
    def __init__(self,X,Y):
        self.x_data = torch.from_numpy(X).type(dtype=torch.float32).resize_((len(X),3,32,32))
        self.y_data = torch.tensor(Y).resize_(len(X),1)

    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

class GH_Dataset_test(data.Dataset): 
    def __init__(self,X,Y):
        self.x_data = torch.from_numpy(X).type(dtype=torch.float32).resize_((10000,3,32,32))
        self.y_data = torch.tensor(Y).resize_(10000,1)

    def __len__(self): 
        return len(self.x_data)
    
    def __getitem__(self, idx): 
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x, y

train_dataset = GH_Dataset_train(train_data_data,train_data_label)
train_loader = data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = GH_Dataset_test(test_data_data,test_data_label)
test_loader = data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

# 모델

In [8]:
class GH(nn.Module):
    def __init__(self):
        super(GH,self).__init__()
        self.l1 = nn.Conv2d(3 ,64,3, padding = 1)

        self.l2 = nn.Conv2d(64,64, 3, padding = 1)
        self.l3 = nn.Conv2d(64,128,3, padding = 1)
        
        self.l4 = nn.Conv2d(128,128,3, padding = 1)
        self.l5 = nn.Conv2d(128,256,3, padding = 1)
        
        self.l6 = nn.Conv2d(256,256,3, padding = 1)
        self.l7 = nn.Conv2d(256,512,3, padding = 1)

        self.l8 = nn.Conv2d(512,512,3, padding = 1)
        self.l9 = nn.Conv2d(512,512,3, padding = 1)

        self.l10 = nn.Linear(2048,1024)   
        self.l11 = nn.Linear(1024,512)
        self.l12 = nn.Linear(512,100)
    
    def GH_Resnet(self,x,residual,k,GH_CONV,GH_CONV2,GH_POOL = True):
      for i in range(k):
        x = F.relu(GH_CONV(x))
        x = GH_CONV(x)
        x += residual
        residual = x
      
      x = F.relu(GH_CONV2(x))
      if GH_POOL:
        x = F.max_pool2d(x,2)
      residual = x
      return x,residual

    def forward(self, x):      #(n,3,32,32)
        x = F.relu(self.l1(x)) #(n,64,32,32)
        residual = x           #residual : (n,64,32,32)

        x,residual = self.GH_Resnet(x,residual,3,self.l2,self.l3) # (n,128,16,16)
        x,residual = self.GH_Resnet(x,residual,3,self.l4,self.l5) # (n,256,8,8)
        x,residual = self.GH_Resnet(x,residual,3,self.l6,self.l7) # (n,512,4,4)
        x,residual = self.GH_Resnet(x,residual,3,self.l8,self.l9) # (n,512,2,2)
        
        
        x = x.view(-1,2048)     #(n,2048)
        x = F.relu(self.l10(x)) #(n,1024)
        x = F.relu(self.l11(x)) #(n,512)
        x = F.relu(self.l12(x)) #(n,100)
        return x                #(n,100)

In [11]:
model = GH()
model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.squeeze(target))
        loss.backward()
        optimizer.step()
        if batch_idx % 100000 == 0:
            print('==================\nTrain Epoch : {} | Loss : {:.6f}'.format(epoch, loss.item()))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, torch.squeeze(target)).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print(f'Test set: Average loss : {test_loss:.4f}, Accuracy : {correct}/{len(test_loader.dataset)}'
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

SyntaxError: ignored

# 모델 학습

In [12]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        
    m, s = divmod(time.time() - since, 60)
    print(f'Total time : {m:.0f}m {s: .0f}s \nModel was trained on {device}!')

Train Epoch : 1 | Loss : 0.000000
Test set: Average loss : 0.1115, Accuracy : 2473/10000(25%)
Training time: 3m 33s
Train Epoch : 2 | Loss : 0.000000
Test set: Average loss : 0.1033, Accuracy : 3061/10000(31%)
Training time: 3m 36s
Train Epoch : 3 | Loss : 0.000000
Test set: Average loss : 0.0996, Accuracy : 3300/10000(33%)
Training time: 3m 36s
Train Epoch : 4 | Loss : 0.000000
Test set: Average loss : 0.0972, Accuracy : 3505/10000(35%)
Training time: 3m 36s
Train Epoch : 5 | Loss : 0.000000
Test set: Average loss : 0.0975, Accuracy : 3555/10000(36%)
Training time: 3m 36s
Train Epoch : 6 | Loss : 0.000000
Test set: Average loss : 0.0993, Accuracy : 3537/10000(35%)
Training time: 3m 36s
Train Epoch : 7 | Loss : 0.000000
Test set: Average loss : 0.1008, Accuracy : 3556/10000(36%)
Training time: 3m 36s
Train Epoch : 8 | Loss : 0.000000
Test set: Average loss : 0.1052, Accuracy : 3494/10000(35%)
Training time: 3m 36s
Train Epoch : 9 | Loss : 0.000000
Test set: Average loss : 0.1081, Accur